<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Dipolar_thrust1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mathplotlib
import numpy as np
from scipy.optimize import minimize_scalar
from matplotlib import pyplot as plt


# Define physical constants
eps0 = 8.854e-12  # vacuum permittivity
mu0 = 1.257e-6  # vacuum permeability
c = 299792458  # speed of light

# Define system parameters
d = 0.1  # distance between dipoles
p1 = np.array([0, 0, 0])  # position of dipole 1
p2 = np.array([0, 0, d])  # position of dipole 2
m = 1  # dipole moment
omega = 2*np.pi*1e9  # frequency of electromagnetic wave


def field_at_point(r, p, m, omega):
    """
    Calculate electric and magnetic fields at a given point in space due to a dipole.
    :param r: position vector of point in space
    :param p: position vector of dipole
    :param m: dipole moment
    :param omega: frequency of electromagnetic wave
    :return: electric and magnetic fields at point r
    """
    # Calculate distance and unit vector between point r and dipole p
    dist = np.linalg.norm(r - p)
    r_hat = (r - p) / dist

    # Calculate electric and magnetic fields
    k = omega / c
    E = (1 / (4 * np.pi * eps0)) * ((3 * np.dot(m, r_hat) * r_hat - m) / dist**3) * np.exp(1j * k * dist)
    B = (mu0 / (4 * np.pi)) * np.cross(E, r_hat)
    
    return E, B

def forces_and_torques(Q, omega, d, m):
    """
    Calculate forces and torques on two dipoles due to electromagnetic fields.
    :param Q: charge on dipoles
    :param omega: frequency of electromagnetic wave
    :param d: distance between dipoles
    :param m: dipole moment
    :return: total force and torque on each dipole
    """
    # Calculate dipole moment
    p = Q * np.array([0, 0, 1])

    # Initialize arrays to store forces and torques
    force1 = np.zeros(3)
    force2 = np.zeros(3)
    torque1 = np.zeros(3)
    torque2 = np.zeros(3)

    # Calculate forces and torques due to electromagnetic fields
    for i in range(3):
        E1, B1 = field_at_point(p1, p2, m, 2*np.pi*omega)
        E2, B2 = field_at_point(p2, p1, m, 2*np.pi*omega)
        force1[i] = np.real(np.dot(E1, p.conj()))
        force2[i] = np.real(np.dot(E2, p.conj()))
        torque1[i] = np.real(np.dot(np.cross(p1 - p2, E1), p.conj()))
        torque2[i] = np.real(np.dot(np.cross(p2 - p1, E2), p.conj()))

    # Calculate total force and torque on each dipole
    F1 = np.sum(force1)
    F2 = np.sum(force2)
    T1 = np.sum(torque1)
    T2 = np.sum(torque2)

    return F1, F2, T1, T2


# Define cost function to minimize (negative thrust)
def cost_function(x):
    # Unpack parameters
    Q, f, d = x
    
    # Calculate dipole moment
    m = Q * d
    
    # Calculate forces and torques due to electromagnetic fields
    force1 = np.zeros(3)
    force2 = np.zeros(3)
    torque1 = np.zeros(3)
    torque2 = np.zeros(3)
    for i in range(3):
        E1, B1 = field_at_point(p1, p2, m, 2*np.pi*f)
        E2, B2 = field_at_point(p2, p1, m, 2*np.pi*f)
        force1[i] = np.real(np.dot(E1, m.conj()))
        force2[i] = np.real(np.dot(E2, m.conj()))
        torque1[i] = np.real(np.dot(np.cross(p1 - p2, E1), m.conj()))
        torque2[i] = np.real(np.dot(np.cross(p2 - p1, E2), m.conj()))
    F1 = np.sum(force1)
    F2 = np.sum(force2)
    T1 = np.sum(torque1)
    T2 = np.sum(torque2)
    
    # Calculate total thrust and return negative for minimization
    return -(F1 + F2)


# Define initial guess and learning rate
x0 = np.array([1e-9, 1e9, 0.1])
learning_rate = 1e-9

# Define range of parameter values
charges = np.logspace(-10, -5, 100)
freqs = np.logspace(9, 11, 100)
distances = np.linspace(0.01, 0.2, 100)

# Calculate thrust for each parameter combination
thrusts = np.zeros((100, 100, 100))
for i, Q in enumerate(charges):
    for j, f in enumerate(freqs):
        for k, d in enumerate(distances):
            F1, F2, T1, T2 = forces_and_torques(Q, f, d, m)
            thrusts[i,j,k] = -(F1 + F2)

# Plot thrust as a function of charge
plt.semilogx(charges, np.max(thrusts, axis=(1,2)))
plt.xlabel("Charge (C)")
plt.ylabel("Thrust (N)")
plt.title("Thrust as a function of charge")
plt.show()

# Plot thrust as a function of frequency
plt.semilogx(freqs, np.max(thrusts, axis=(0,2)))
plt.xlabel("Frequency (Hz)")
plt.ylabel("Thrust (N)")
plt.title("Thrust as a function of frequency")
plt.show()

# Plot thrust as a function of distance between dipoles
plt.plot(distances, np.max(thrusts, axis=(0,1)))
plt.xlabel("Distance between dipoles (m)")
plt.ylabel("Thrust (N)")
plt.title("Thrust as a function of distance between dipoles")
plt.show()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement mathplotlib (from versions: none)
ERROR: No matching distribution found for mathplotlib
